In [0]:
from pyspark.sql.functions import col, expr, sequence, to_date, year, month, dayofmonth, dayofweek, weekofyear, quarter, date_format
from pyspark.sql import SparkSession

In [0]:
%run "./CONSTANTS"

In [0]:
%run "./UTIL_FUNCTIONS"

In [0]:
start_date = "2000-01-01"
end_date   = "2030-12-31"

In [0]:

df_dates = (spark.sql(f"SELECT sequence(to_date('{start_date}'), to_date('{end_date}'), interval 1 day) as calendar_dates")
                 .withColumn("date", expr("explode(calendar_dates)"))
                 .drop("calendar_dates"))

date_dim = (df_dates
    .withColumn("year", year("date"))
    .withColumn("quarter", quarter("date"))
    .withColumn("month", month("date"))
    .withColumn("week_of_year", weekofyear("date"))
    .withColumn("day_of_month", dayofmonth("date"))
    .withColumn("day_of_week", dayofweek("date"))   # 1 = Sunday in Spark
    .withColumn("day_name", date_format(col("date"), "EEEE"))  # Monday, Tuesday...
    .withColumn("month_name", date_format(col("date"), "MMMM")) # January, February...
)

display(date_dim)

In [0]:
write_to_data_lake(date_dim, 'delta', 'owerwrite', '{curated_path}/COM_DATE_D', '')